In [1]:
import pandas as pd
import ast

In [2]:
# Importing raw data

with open("2018_ep_demo.txt", encoding='utf-8-sig') as f:                    # Change the file name here if necessary
    data = ast.literal_eval(f.read())

In [3]:
ep_count_start = len(data)
print('Found %d accounts.' % (ep_count_start))

data

Found 1 accounts.


[{'full_name': 'Steve Rogers',
  'profile_photo': 'https://s3-eu-west-1.amazonaws.com/cdn.expa.aiesec.org/gis-img/missing_profile_s.svg',
  'email': 'captain_america@inbound.aiesec.org',
  'status': 'applied',
  'created_at': '2018-12-31T00:00:00Z',
  'managed_opportunities_count': 0,
  'managed_opportunities': {'edges': []},
  'managers': [{'full_name': 'Peggy Carter',
    'profile_photo': 'https://s3-eu-west-1.amazonaws.com/cdn.expa.aiesec.org/gis-img/missing_profile_s.svg',
    'id': '0000001',
    'email': 'agent_carter@inbound.aiesec.org'}],
  'current_experiences': [],
  'phone': '12345678',
  'id': '0000000',
  'first_name': 'Steve',
  'last_name': 'Rogers',
  'dob': '1918-07-04',
  'permissions': {'can_update': 'true'},
  'person_profile': {'selected_programmes': [5, 1], 'id': '0000000'},
  'organisation_type': 'null',
  'referral_type': 'Friend'}]

In [4]:
# Removing unneeded columns

remove = ("current_experiences","first_name","last_name","managed_opportunities","managed_opportunities_count",
          "organisation_type","permissions","profile_photo")
for i in range(len(data)):
    for key in remove:
        data[i].pop(key)

In [5]:
# Cleaning up selected programme data
for i in range(len(data)):
    prog_list = []
    if data[i]['person_profile'] != 'null':
        for j in range(len(data[i]['person_profile']['selected_programmes'])):
            prog_list.append(str(data[i]['person_profile']['selected_programmes'][j]))
    prog = ",".join(prog_list)
    prog = prog.replace('1', 'GV')
    prog = prog.replace('2', 'GT')
    prog = prog.replace('5', 'GE')
    data[i]['person_profile'] = prog

In [6]:
# Cleaning up EP manager data

for i in range(len(data)):
    epm_list = []
    for j in range(len(data[i]['managers'])):
        epm_list.append(data[i]['managers'][j]['full_name'])
    epm = ", ".join(epm_list)
    data[i]['managers'] = epm

In [7]:
# Cleaning up empty date of birth and phone number data

for i in range(len(data)):
    dob = data[i]['dob']
    if dob == 'null':
        dob = dob.replace('null', '')
        data[i]['dob'] = dob
    phone = data[i]['phone']
    if phone == 'null':
        phone = phone.replace('null', '')
        data[i]['phone'] = phone
    else: 
        phone = str(phone)
        data[i]['phone'] = phone

In [8]:
# Cleaning up account creation date data

for i in range(len(data)):
    data[i]['created_at'] = data[i]['created_at'][:10]

In [9]:
# Converting all data into DataFrame

EP_Data = pd.DataFrame(data)

In [10]:
# Removing deleted accounts

for i in range(len(data)):
    if EP_Data['status'][i] == 'deleted':
        EP_Data = EP_Data.drop([i], axis=0)

EP_Data = EP_Data.reset_index()
ep_count_new = len(EP_Data)
deleted_count = ep_count_start - ep_count_new
print('%d deleted accounts removed.' % (deleted_count))

0 deleted accounts removed.


In [11]:
# Rearranging and tidying up the columns (OCD lol)

EP_Data = EP_Data[['id','status','full_name','person_profile','managers','phone','email','dob','referral_type','created_at']]

rename = {'id':'EP_ID',
          'status':'Status',
          'full_name':'Name',
          'person_profile':'Interested',
          'managers': 'Managers',
          'phone':'Phone',
          'email':'Email',
          'dob':'Birthdate',
          'referral_type':'Referral',
          'created_at':'Created'}

EP_Data.rename(columns=rename, inplace=True) 

In [12]:
print('EP Count: %d' % (ep_count_new))

EP Count: 1


In [13]:
# Export to Excel
#EP_Data.to_excel("EXPA.xlsx")
EP_Data

,EP_ID,Status,Name,Interested,Managers,Phone,Email,Birthdate,Referral,Created
0,0000000,applied,Steve Rogers,"GE,GV",Peggy Carter,12345678,captain_america@inbound.aiesec.org,1918-07-04,Friend,2018-12-31


In [14]:
# Export to CSV
#EP_Data.to_csv("EXPA.csv")